In [2]:
'''
!pip uninstall -y numpy tensorflow tensorflow-estimator tensorflow-privacy tensorflow-probability -q

!pip install tensorflow==2.15.0 -q
!pip install tensorflow-privacy -q
!pip install tensorflow-probability==0.23.0 -q'''

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 121.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 37.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires tensorflow-probability>=0.13.0, which is not installed.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
tensorstore 0.1.74 requires ml_dtypes>=0.3.1, but you have ml-dtypes 0.2.0 which is incompatible.
ydf 0.12.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 4.25.8 whic

In [5]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras

from tensorflow_privacy.privacy.optimizers.dp_optimizer_keras import DPKerasSGDOptimizer
from tensorflow_privacy.privacy.analysis.compute_dp_sgd_privacy_lib import compute_dp_sgd_privacy

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler

In [9]:
df = pd.read_csv("athletes_v2.csv")
X = df.drop(columns=["total_lift", "candj", "snatch", "deadlift","backsq"])
y = df["total_lift"]

In [13]:
# Normalize numeric columns
numeric = X.select_dtypes(include=np.number).columns
X[numeric] = (X[numeric] - X[numeric].mean()) / X[numeric].std()

In [ ]:
# model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

In [17]:
n_train = len(X_train)
n_train

24649

In [19]:
# Set hyperparameters
noise_multiplier = 1.1  # moderate privacy
l2_norm_clip = 1.0      # clip gradients to this norm
batch_size = 32         # smaller batches = better privacy
learning_rate = 0.01    # often need lower LR with DP
epochs = 50             # may need more due to noise

# calculate delta based on dataset size
delta = 1 / n_train

# convert tensr flow to datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=n_train)
train_dataset = train_dataset.batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_dataset = test_dataset.batch(batch_size)

from os import access
# build model with DP optimizer
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(1)   # regression output
])

# create DP optimizer
dp_optimizer = DPKerasSGDOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=batch_size,   # process each example separately
    learning_rate=learning_rate
)

In [27]:
# compile with per-example loss
model.compile(optimizer=dp_optimizer,
              loss=tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE),
              metrics=['mae']
              )

In [28]:
# train
history = model.fit(train_dataset,
                    epochs=epochs,
                    validation_data=test_dataset,
                    verbose=1)

Epoch 1/50
770/770 [==============================] - 4s 4ms/step - loss: 1080962.0000 - mae: 1002.5010 - val_loss: 1012960.0625 - val_mae: 968.6173
Epoch 2/50
770/770 [==============================] - 2s 3ms/step - loss: 839027.8125 - mae: 877.0268 - val_loss: 604086.1250 - val_mae: 740.2415
Epoch 3/50
770/770 [==============================] - 2s 3ms/step - loss: 320368.7812 - mae: 503.6274 - val_loss: 98882.0703 - val_mae: 260.5882
Epoch 4/50
770/770 [==============================] - 2s 3ms/step - loss: 60348.8008 - mae: 194.4201 - val_loss: 50761.6055 - val_mae: 176.6268
Epoch 5/50
770/770 [==============================] - 3s 4ms/step - loss: 45985.6016 - mae: 168.2467 - val_loss: 43940.8711 - val_mae: 164.1131
Epoch 6/50
770/770 [==============================] - 3s 3ms/step - loss: 41235.0273 - mae: 159.5553 - val_loss: 40406.6758 - val_mae: 157.0315
Epoch 7/50
770/770 [==============================] - 3s 3ms/step - loss: 38728.3555 - mae: 154.5964 - val_loss: 38166.4258 - va

In [33]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [37]:
# evaluate
predictions = model.predict(X_test)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f"mae: {mae:.2f}")
print(f"R²: {r2:.3f}")

193/193 [==============================] - 0s 788us/step
mae: 133.60
R²: 0.610


In [39]:
# calculate epsilon (privacy spent)
epsilon, _ = compute_dp_sgd_privacy(
    n=n_train,
    batch_size=batch_size,
    noise_multiplier=noise_multiplier,
    epochs=epochs,
    delta=delta
)
print(f"Privacy guarantee: ε = {epsilon:.2f} at δ = {delta:.2e}")


Privacy guarantee: ε = 1.10 at δ = 4.06e-05


- epsilon (privacy guarantee) > 1 is good!
- δ is very small (0.0000406) which is good. This means there is only a 0.00406% chance of privacy breach.
